<a href="https://colab.research.google.com/github/SushovitNanda/SemEval-Food-Hazards/blob/main/Bert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Import necessary libraries
%%capture
!pip install transformers
!pip install datasets --upgrade
#!pip install torch
import os
import pandas as pd
!pip install evaluate
import evaluate
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
import torch
from transformers import RobertaTokenizer, RobertaForSequenceClassification, Trainer, TrainingArguments, BertTokenizer, BertForSequenceClassification
from transformers import DataCollatorWithPadding
from datasets import Dataset
import warnings
warnings.filterwarnings("ignore")

# Disable W&B logging
os.environ["WANDB_MODE"] = "disabled"

In [2]:
# Load the dataset
train = pd.read_csv('incidents_train.csv')

# Combine 'title' and 'text' columns to create input data
train['input_text'] = train['title'] + " " + train['text']

# Encode the target labels
label_encoder = LabelEncoder()
train['label'] = label_encoder.fit_transform(train['hazard-category'])

# Train-test split using stratification
train_df, test_df = train_test_split(
    train[['input_text', 'label']],
    test_size=0.2,
    stratify=train['label'],
    random_state=42
)

# Convert the dataframes into Hugging Face's Dataset format
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

# Load the RoBERTa tokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

# Define a tokenization function
def tokenize_function(example):
    return tokenizer(example['input_text'], truncation=True)

# Tokenize the datasets
train_dataset = train_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)

# Set up a data collator to pad inputs dynamically
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# Load RoBERTa model for sequence classification
num_labels = len(label_encoder.classes_)
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=num_labels)


Map:   0%|          | 0/4065 [00:00<?, ? examples/s]

Map:   0%|          | 0/1017 [00:00<?, ? examples/s]

In [3]:
# Load the F1 metric and specify macro averaging
f1_metric = evaluate.load("f1")

# Define the compute_metrics function to maximize F1 macro average
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    # Compute the F1 macro average
    f1 = f1_metric.compute(predictions=predictions, references=labels, average="macro")
    return {"f1": f1["f1"]}

# Update training arguments to focus on F1 score
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=1e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=5,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="f1"  # Use F1 score as the metric for saving best model
)

# Redefine the Trainer with the updated compute_metrics
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics  # Use F1 macro for evaluation
)

# Train the model
trainer.train()

Epoch,Training Loss,Validation Loss,F1
1,0.250300,0.365778,0.469653
2,0.259600,0.282834,0.495599
3,0.197200,0.259536,0.504963
4,0.268400,0.260000,0.612580
5,0.119500,0.261916,0.687645


TrainOutput(global_step=2545, training_loss=0.3238282489870293, metrics={'train_runtime': 2134.9731, 'train_samples_per_second': 9.52, 'train_steps_per_second': 1.192, 'total_flos': 5340634077078972.0, 'train_loss': 0.3238282489870293, 'epoch': 5.0})

In [4]:
# Evaluate the model
predictions = trainer.predict(test_dataset)
preds = predictions.predictions.argmax(-1)

# Classification report
target_names = label_encoder.classes_
print(classification_report(test_df['label'], preds, target_names=target_names))

                                precision    recall  f1-score   support

                     allergens       0.95      0.99      0.97       371
                    biological       0.99      0.99      0.99       348
                      chemical       0.85      1.00      0.92        57
food additives and flavourings       1.00      0.40      0.57         5
                foreign bodies       0.94      1.00      0.97       112
                         fraud       0.88      0.69      0.77        74
                     migration       0.00      0.00      0.00         1
          organoleptic aspects       1.00      0.36      0.53        11
                  other hazard       0.48      0.41      0.44        27
              packaging defect       1.00      0.55      0.71        11

                      accuracy                           0.94      1017
                     macro avg       0.81      0.64      0.69      1017
                  weighted avg       0.94      0.94      0.94 